In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision

In [ ]:
import set_data as sd

# load train set, test set

X_train, Y_train, X_test, Y_test = sd.set_data("data/combined_total_data_1")
print(X_train[0], Y_train[0])

x_train = Variable(X_train)
y_train = Variable(Y_train)


In [ ]:
# regression model

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(28, 64) #input dim = 28, hidden = 128
        self.drop1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 10)
        self.drop2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(10, 5)
        self.drop3 = nn.Dropout(0.1)
        self.fc4 = nn.Linear(5, 1)
        

    def forward(self, x):
        a = F.relu(self.fc1(x))
        a = self.drop1(a)
        a = F.relu(self.fc2(a))
        a = self.drop2(a)
        a = F.relu(self.fc3(a))
        a = self.drop3(a)
        a = self.fc4(a)
        
        return a

In [ ]:
# train

epochs = 30
learningrate = 0.01

model = RegressionModel()
criterion = nn.SmoothL1Loss(size_average = True) 
optimizer = torch.optim.Adam(model.parameters(), lr=learningrate)

print('\nStart Training...\n')

loss_graph = []
for epoch in range(epochs+1):
    # forward pass
    pred_y = model(x_train.float())
    

    # compute and print loss
    loss = criterion(pred_y.double(), y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    loss_graph.append(loss.item())
    
    if epoch % 10 == 0:
        print ('epoch [{}/{}], loss {}'.format(epoch, epochs, loss.item())) # or loss item
    

In [ ]:
torch.save(model.state_dict(), "./lenna0.pth")

In [ ]:
fig = plt.figure()
plt.plot(range(epochs+1), loss_graph, label = 'loss', color = 'black')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid(linestyle = '--', color = 'lavender')
plt.show()

In [ ]:
x_test = Variable(X_test)
y_test = Variable(Y_test)

model.eval()
y_pred = model(x_test.float())
after_train = criterion(y_pred.double(), y_test) 
print('Test loss after Training' , after_train.item())

In [ ]:
new_model= RegressionModel()
new_model.load_state_dict(torch.load('./lenna0.pth'))
    
new_model.eval()